<a href="https://colab.research.google.com/github/thunni-noi/jp-vtube-livetranslator/blob/main/speech-to-text/STT_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 27 05:02:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install -q unidic-lite 
!pip install -q mecab-python3 
!pip install -q datasets==1.18.3 
!pip install -q torchaudio 
!pip install -q transformers==4.17.0 
!pip install -q librosa 
!pip install -q jiwer 
!pip install -q pykakasi 
!pip install -q IPython
!pip install -q --upgrade IPython 
exit() #restart runtime after import datasets package!

     |████████████████████████████████| 47.4 MB 2.0 MB/s 
     |████████████████████████████████| 574 kB 5.5 MB/s 
     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 65.2 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 140 kB 72.0 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 144 kB 68.4 MB/s 
     |████████████████████████████████| 271 kB 64.9 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 49.7 MB/s 
     |████████████████████████████████| 880 kB 59.7 MB/s 
     |████████████████████████████████| 50 kB 3.4 MB/s 
     |████████████████████████████████| 2.4 MB 5.2 MB/s 
     |████████████████████████████████| 793 kB 4.7 MB/s 
     |███████████████████

In [1]:
from huggingface_hub import notebook_login
notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:587: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.8.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [2]:
#all debugging tools

import os
import IPython
import re
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import IPython.display as ipd
import numpy as np
import requests
from tqdm.notebook import tqdm_notebook
import time
from datetime import timedelta

def line_noti_msg(msg): #sent line noti
  line_noti_url = 'https://notify-api.line.me/api/notify'
  line_noti_token = '{LINE_NOTI_TOKEN}'
  headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+line_noti_token}
  r = requests.post(line_noti_url, headers=headers, data = {'message':msg})

def show_random_transcript(dataset, num_examples=10, unwanted=[]): 
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    dataset = dataset.remove_columns(unwanted)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
  
def play_random(ds): #play random speech with a transcript
  rand_int = random.randint(0, len(ds)-1)
  print(ds[rand_int]["sentence"])
  return(ipd.Audio(data=ds[rand_int]["audio"]['array'], autoplay=True, rate=16000))

def play_random_array(ds): #play random speech with a transcript
  rand_int = random.randint(0, len(ds)-1)
  print(ds[rand_int]["target_text"])
  return(ipd.Audio(data=np.asanyarray(ds[rand_int]["speech"]), autoplay=False, rate=16000))

def gen_excel(data, name = str('excel'), override = bool(False)): #Datasets to excel
  if type(data) != 'pandas.core.frame.DataFrame' : 
    temp = pd.DataFrame(data)
    data = temp
    del temp
  path = '/content/excel/'
  if not os.path.exists(path) :  
    os.makedirs(path)
    print('----- creating new path ----')
  tempname = name
  tries = 0
  if not override : 
    while os.path.exists(f'{path}{tempname}.xlsx') : 
      tempname = str(tempname) + str(tries)
      tries += 1
    print(f'---- creating new excel file name : {tempname} ---- ')
    del tries
  else :
    print(f'---- overriding excel file name : {tempname} ----')

  filename = tempname + str('.xlsx')
  data.to_excel(f'{path}{filename}', sheet_name='Dataframe', header = True)
  return f'{path}{filename}' #return as excel pth

def audio_info(ds) :
  cur_ds = ds[random.randint(0, len(ds)-1)] #select random speech
  print ('Target text : ', cur_ds['sentence'])
  print('Input array shape : ', cur_ds['audio']['array'].shape)
  print('Sampling rate : ', cur_ds['audio']['sampling_rate'])

def pkl_concat(pkl_path_list: list):
  temp_df_list = []
  for pkl_path in pkl_path_list:
    temp_df = pd.read_pickle(pkl_path)
    temp_df_list.append(temp_df)
  return pd.concat(temp_df_list)


In [3]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category = DeprecationWarning)
#i still don't know what caused this so fuc it

In [4]:
#clean the data!!!!!
import pykakasi #package to convert japanese to hiragana
import MeCab
import torchaudio
import json
import re
vocabPath = '/content/vocab.json'
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\‘\”\�‘、。．！，_・―─~｢｣『』\\\\※\[\]\{\}「」〇？…]' #special characters that isn't in japanese
wakati = MeCab.Tagger("-Owakati")
kakasi = pykakasi.kakasi()
#hiragana
kakasi.setMode('J', 'H') #kanji
kakasi.setMode('K', 'H') #katakana
conv = kakasi.getConverter()
FULLWIDTH_TO_HALFWIDTH = str.maketrans(
  '　０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ！゛＃＄％＆（）＊＋、ー。／：；〈＝〉？＠［］＾＿‘｛｜｝～',
  ' 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&()*+,-./:;<=>?@[]^_`{|}~',
)
emoticon_pattern = re.compile("["
        u"\U00012600-\U000126FF"  #miscellaneous symbol
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

symbol_pattern = re.compile('['
        r'\u2190-\u21FF' #arrow
        r'\u27F0-\u27FF' #arrow 2
        r'\u2900-\u297F' #arrow 3
        r'\u1F800-\u1F8FF' #arrow 4
        r'\u1F800-\u1F8FF' #arrow 5
        r'\u2000-\u206F' #punctuation
        r'\u20A0-\u20CF' #currency
        r'\u0391-\u03C9' #greek
        r'\u2200-\u22FF' #math operator
        r'\u2600-\u26FF' #misc
        r'\u2500-\u257F' #box 
        r'\u25A0-\u25FF' #shapes
        r'\u301C' #idk what's dis
        ']+', re.UNICODE)

def preprocessData(batch):
  batch['sentence'] = batch['sentence'].translate(FULLWIDTH_TO_HALFWIDTH)
  batch["sentence"] = re.sub(emoticon_pattern,' ', batch["sentence"])
  batch["sentence"] = re.sub(symbol_pattern,' ', batch["sentence"])
  batch["sentence"] = re.sub(chars_to_ignore_regex,' ', batch["sentence"]).lower()  #remove special char
  batch["sentence"] = wakati.parse(batch["sentence"]) #space
  batch["sentence"] = conv.do(batch["sentence"]) #convert to hiragana
  batch["sentence"] = " ".join(batch["sentence"].split())+" " #remove space

  return batch

def createVocabList(dataset):
    def extract_all_chars(batch):
        all_text = " ".join(batch["sentence"])
        vocab = list(set(all_text))
        return {"vocab": [vocab], "all_text": [all_text]}

    
    dataset=dataset.map(preprocessData)
    vocab_train = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names) #concat all text
    vocab_list = list(set(vocab_train["vocab"][0]) )      #convert to set
    vocab_list = sorted(vocab_list)
    vocab_dict = {v: k for k, v in enumerate(vocab_list)}   # convert to dict
    vocab_dict['|'] = vocab_dict[' ']
    del vocab_dict[' ']
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)
    print(vocab_dict)
    print(vocab_list)

    with open(vocabPath, 'w')as vocab_file:
        json.dump(vocab_dict, vocab_file)

    print(vocab_dict)

def audio_resample(dataset):
  dataset = dataset.cast_column('audio', Audio(sampling_rate=16_000))
  assert dataset[0]['audio']['sampling_rate'] == 16000
  return dataset


#prepare for training
def audio_to_array(batch):
  array, sampling_rate = torchaudio.load(batch['path'])
  batch['speech'] = torchaudio.functional.resample(array, sampling_rate, 16000)[0].numpy()
  batch['target_text'] = batch['sentence']
  return batch

def filterLen(batch):
  array, sampling_rate = torchaudio.load(batch['path'])
  return array.shape[1]/sampling_rate<11.0

In [5]:
#load datasets
import pandas as pd
from datasets import Dataset, load_dataset
def load_datasets_from_pickle(pkl_parent_folder):

  all_pkl_list = []

  temp_all_pkl_list =[pkl_list for pkl_list in os.listdir(pkl_parent_folder) if 'clean' in pkl_list]
  
  for temp_path in temp_all_pkl_list : all_pkl_list.append(os.path.join(pkl_parent_folder, temp_path))
  mixed_dataframe = pkl_concat(all_pkl_list)
  
  raw_dataset = Dataset.from_pandas(mixed_dataframe).remove_columns(['__index_level_0__'])
  split_datasets = raw_dataset.train_test_split(train_size=0.85, seed = 48)
  line_noti_msg('Finished loading datasets!')
  return split_datasets

In [6]:
from datasets import concatenate_datasets
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, AutoProcessor, AutoModelForCTC
def prepare_dataset(batch):    
    batch["input_values"] = processor(batch["speech"], sampling_rate=16000).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch
  
def prepare_dataset_for_training(split_datasets):
  #assert str(ds_test.features) == str(ds_train.features), f'your code is fucked, fix it pls'
  #backup incase shit got messed up
  split_datasets = split_datasets.map(preprocessData).filter(lambda batch: bool(batch['sentence']))
  split_datasets = split_datasets.filter(filterLen, num_proc=14)
  split_datasets = split_datasets.map(audio_to_array)
  split_datasets = split_datasets.map(prepare_dataset, batch_size=8, num_proc=12, batched=True,remove_columns=split_datasets.column_names)

  line_noti_msg('Finished preparing datasets!')
  return split_datasets



BEGIN TRAINING LESS GOOOOOOOOOOOOO

In [7]:
#@title DataCollator
import torch
from datasets import load_metric
#datacollator
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


wer_metric = load_metric('wer')
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [8]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer, AutoModelForCTC

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("ttop324/wav2vec2-live-japanese")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = AutoProcessor.from_pretrained("ttop324/wav2vec2-live-japanese")
model = AutoModelForCTC.from_pretrained("ttop324/wav2vec2-live-japanese", mask_time_prob = 0).cuda()
data_collator = DataCollatorCTCWithPadding(processor=processor, padding = True)
model.freeze_feature_extractor()

checkpoint_output_path = '/content/checkpoints'
if not os.path.exists(checkpoint_output_path) : os.makedirs(checkpoint_output_path)

training_args = TrainingArguments(
  output_dir=checkpoint_output_path,  
  group_by_length=True,
  per_device_train_batch_size=3,
  gradient_accumulation_steps=2,
  per_device_eval_batch_size=2,
  num_train_epochs=50,
  fp16=True,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=1,
    push_to_hub=True
)

'''
training_args = TrainingArguments(
  output_dir='./temporary_model_pretrained',
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)
'''

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at ttop324/wav2vec2-live-japanese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.masked_spec_embed']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


'\ntraining_args = TrainingArguments(\n  output_dir=\'./temporary_model_pretrained\',\n  group_by_length=True,\n  per_device_train_batch_size=8,\n  evaluation_strategy="steps",\n  num_train_epochs=30,\n  fp16=True,\n  gradient_checkpointing=True,\n  save_steps=500,\n  eval_steps=500,\n  logging_steps=500,\n  learning_rate=1e-4,\n  weight_decay=0.005,\n  warmup_steps=1000,\n  save_total_limit=2,\n)\n'

In [9]:
try:
  pickle_folder_path = '/content/drive/MyDrive/livetranslator/pickle'
  split_dataset = load_datasets_from_pickle(pickle_folder_path)
  train_ds = split_dataset['train']
  test_ds = split_dataset['test']
  train_ds = prepare_dataset_for_training(train_ds)
  test_ds = prepare_dataset_for_training(test_ds)

  trainer = Trainer(
      model=model,
      data_collator=data_collator,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_ds,
      eval_dataset=test_ds,
      tokenizer=processor.feature_extractor,
  )
except Exception as error_msg:
  line_noti_msg(f'Encountered error! {error_msg}')

Parameter 'function'=<function preprocessData at 0x7f5183b689e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

...

Clean file runs/May26_17-27-40_3b120d93ba05/1653586261.552507/events.out.tfevents.1653586261.3b120d93ba05.470.…

Clean file training_args.bin:  34%|###4      | 1.00k/2.92k [00:00<?, ?B/s]

Clean file runs/May26_17-27-40_3b120d93ba05/events.out.tfevents.1653586261.3b120d93ba05.470.0:   4%|3         …

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Using amp half precision backend


In [ ]:
%%capture var
try:
  start_time = time.monotonic()
  line_noti_msg('Started training!!')
  trainer.train()
  line_noti_msg('Finished training!')
  end_time = time.monotonic()
  line_noti_msg(f'Total execution time : {timedelta(seconds=end_time - start_time)}')
except Exception as error_msg:
  line_noti_msg(f'Encountered error! {error_msg}')

***** Running training *****
  Num examples = 742
  Num Epochs = 50
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 2
  Total optimization steps = 6200
***** Running Evaluation *****
  Num examples = 132
  Batch size = 2
Saving model checkpoint to /content/checkpoints/checkpoint-124
Configuration saved in /content/checkpoints/checkpoint-124/config.json
Model weights saved in /content/checkpoints/checkpoint-124/pytorch_model.bin
Feature extractor saved in /content/checkpoints/checkpoint-124/preprocessor_config.json
Feature extractor saved in /content/checkpoints/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 132
  Batch size = 2
Saving model checkpoint to /content/checkpoints/checkpoint-248
Configuration saved in /content/checkpoints/checkpoint-248/config.json
Model weights saved in /content/checkpoints/checkpoint-248/pytorch_model.bin
Feature extractor saved in /co

In [ ]:
var.show()

In [ ]:
processor.save_pretrained('wav2vec2-model-jp-2')
model.save_pretrained('wav2vec2-model-jp-2')

In [ ]:
trainer.push_to_hub('thunninoi/wav2vec2-jp-vtube-finetuned')